# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 24 2022 2:31PM,240421,18,CLRSPC-21119958,"ClearSpec, LLC",Released
1,Jun 24 2022 2:26PM,240420,19,DEX0007058,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 24 2022 2:26PM,240420,19,DEX0007059,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 24 2022 2:26PM,240420,19,DEX0007060,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jun 24 2022 2:26PM,240420,19,DEX0007061,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 24 2022 2:26PM,240420,19,DEX0007062,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jun 24 2022 2:20PM,240419,18,CLRSPC-21115785,"ClearSpec, LLC",Released
7,Jun 24 2022 1:47PM,240403,20,8169256-FLS-BO,"ACI Healthcare USA, Inc.",Executing
8,Jun 24 2022 1:47PM,240403,20,ACI-8182208,"ACI Healthcare USA, Inc.",Executing
9,Jun 24 2022 1:47PM,240403,20,8183907,"ACI Healthcare USA, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,240404,Released,35
7,240405,Executing,2
8,240419,Released,1
9,240420,Released,5
10,240421,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240404,NaN,35.0
240405,2.0,NaN
240419,NaN,1.0
240420,NaN,5.0
240421,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240348,0.0,1.0
240352,0.0,1.0
240355,0.0,1.0
240385,0.0,1.0
240386,0.0,36.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240348,0,1
240352,0,1
240355,0,1
240385,0,1
240386,0,36


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240348,0,1
1,240352,0,1
2,240355,0,1
3,240385,0,1
4,240386,0,36


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240348,,1
1,240352,,1
2,240355,,1
3,240385,,1
4,240386,,36


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 24 2022 2:31PM,240421,18,"ClearSpec, LLC"
1,Jun 24 2022 2:26PM,240420,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
6,Jun 24 2022 2:20PM,240419,18,"ClearSpec, LLC"
7,Jun 24 2022 1:47PM,240403,20,"ACI Healthcare USA, Inc."
10,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation
45,Jun 24 2022 11:38AM,240405,20,"ACI Healthcare USA, Inc."
47,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation
83,Jun 24 2022 9:17AM,240385,12,"SD Head USA, LLC"
84,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.
85,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 24 2022 2:31PM,240421,18,"ClearSpec, LLC",,1
1,Jun 24 2022 2:26PM,240420,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,5
2,Jun 24 2022 2:20PM,240419,18,"ClearSpec, LLC",,1
3,Jun 24 2022 1:47PM,240403,20,"ACI Healthcare USA, Inc.",3,
4,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,,35
5,Jun 24 2022 11:38AM,240405,20,"ACI Healthcare USA, Inc.",2,
6,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation,,36
7,Jun 24 2022 9:17AM,240385,12,"SD Head USA, LLC",,1
8,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,,1
9,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 2:31PM,240421,18,"ClearSpec, LLC",1,
1,Jun 24 2022 2:26PM,240420,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5,
2,Jun 24 2022 2:20PM,240419,18,"ClearSpec, LLC",1,
3,Jun 24 2022 1:47PM,240403,20,"ACI Healthcare USA, Inc.",,3
4,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,35,
5,Jun 24 2022 11:38AM,240405,20,"ACI Healthcare USA, Inc.",,2
6,Jun 24 2022 9:22AM,240386,10,ISDIN Corporation,36,
7,Jun 24 2022 9:17AM,240385,12,"SD Head USA, LLC",1,
8,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,1,
9,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 2:31PM,240421,18,"ClearSpec, LLC",1,
1,Jun 24 2022 2:26PM,240420,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5,
2,Jun 24 2022 2:20PM,240419,18,"ClearSpec, LLC",1,
3,Jun 24 2022 1:47PM,240403,20,"ACI Healthcare USA, Inc.",,3
4,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,35,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 2:31PM,240421,18,"ClearSpec, LLC",1.0,NaN
1,Jun 24 2022 2:26PM,240420,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5.0,NaN
2,Jun 24 2022 2:20PM,240419,18,"ClearSpec, LLC",1.0,NaN
3,Jun 24 2022 1:47PM,240403,20,"ACI Healthcare USA, Inc.",NaN,3.0
4,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,35.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 2:31PM,240421,18,"ClearSpec, LLC",1.0,0.0
1,Jun 24 2022 2:26PM,240420,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5.0,0.0
2,Jun 24 2022 2:20PM,240419,18,"ClearSpec, LLC",1.0,0.0
3,Jun 24 2022 1:47PM,240403,20,"ACI Healthcare USA, Inc.",0.0,3.0
4,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,35.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,480790,71.0,0.0
12,240385,1.0,0.0
16,721055,3.0,0.0
18,480840,2.0,0.0
19,240420,5.0,0.0
20,480808,0.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,480790,71.0,0.0
1,12,240385,1.0,0.0
2,16,721055,3.0,0.0
3,18,480840,2.0,0.0
4,19,240420,5.0,0.0
5,20,480808,0.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,71.0,0.0
1,12,1.0,0.0
2,16,3.0,0.0
3,18,2.0,0.0
4,19,5.0,0.0
5,20,0.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,71.0
1,12,Released,1.0
2,16,Released,3.0
3,18,Released,2.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Executing,0.0,0.0,0.0,0.0,0.0,5.0
Released,71.0,1.0,3.0,2.0,5.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Executing,0.0,0.0,0.0,0.0,0.0,5.0
1,Released,71.0,1.0,3.0,2.0,5.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Executing,0.0,0.0,0.0,0.0,0.0,5.0
1,Released,71.0,1.0,3.0,2.0,5.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()